### for debug production app

In [154]:
# choose execl 
def choose_file():
    
    return 'FFG-November-Options-Activity' + '.xlsx'


In [155]:
# input shares dates
def get_date():
    return datetime.date(2018, 01, 24)  # return datetime.datetime.now()


In [156]:
# read xls into pandas table
def create_table(file_name):
    tables_list = pd.read_excel( file_name, sheetname=None)
    
    # get last worksheet from list
    table = tables_list[tables_list.keys()[-1]]
    
    # clear nan rows
    table = table[pd.isnull(table.SYMBL) == False]
    
    # extract existing found-codes
    symbl = table.SYMBL.dropna().unique()
    
    return table, symbl  # return table and list of found market symbols


In [189]:
# read and preprocess shares from yahoo on current date
def read_shares(symbl, start, end):
    # start = end = datetime.datetime.now()
    # start = end = get_date()
    shares_panel = web.DataReader(symbl, 'yahoo', start, end)
    
    # convert Panel to the Multyindexed frame
    shares = shares_panel.to_frame()
    
    # remove yesterday (start_date -1) from multyindex df
    yesterday = start - timedelta(days=1)
    #print (yesterday)
    #print shares
    shares.drop(yesterday, level=0, axis=0, inplace=True)
    #print shares

  
    # remove index level 0 (data)
    shares.index = shares.index.droplevel(0)
    
    # convert index into column
    shares.reset_index(inplace=True)
    
    shares.rename(columns={'minor': 'symbl'}, inplace=True)
    
    # insert column
    shares['Current Date'] = start
    
    # remove column
    shares.drop(['Volume'], axis=1, inplace=True)
    
    return shares


In [158]:
# create output xls from table and shares dataframes
def create_output(table, shares):
    # create table
    table = pd.merge(left=table, right=shares, left_on='SYMBL', right_on='symbl')
    
    # delete 2-nd `symbol`
    table.drop('symbl', axis=1, inplace=True)
    
    # reformat date columns to m/d/y
    # table['Current Date'] = table['Current Date'].map(lambda x: x.strftime(format='%m/%d/%Y'))
    # table['EXP DATE'] = table['EXP DATE'].strftime('%m/%d/%Y')
    # table['EXP DATE'] = table['EXP DATE'].dt.strftime('%m/%d/%Y')
    
    # reformat prices to 2 dec.digits
    round_2 = lambda x: round(x, 2)
    table.Open = table.Open.map(round_2)
    table.Low = table.Low.map(round_2)
    table.High = table.High.map(round_2)
    table.Close = table.Close.map(round_2)
    table['Adj Close'] = table['Adj Close'].map(round_2)
    
    # reorder columns
    order = list(table.columns)[:-6]  # exept new columns
    new_order = order[:len(order)] + ['Current Date', 'Open', 'High', 'Low', 'Close', 'Adj Close']
    new_table = table[new_order]
    
    # reorder SEQ column through ass
    new_table.SEQ = range(1, new_table.SEQ.shape[0] + 1)
    
    # write new xls table
    new_table.to_excel('result_' + file_name, index=False)


### Start here

In [185]:
# main part
import os
import pandas as pd
import pandas_datareader.data as web
from datetime import date
from datetime import datetime
import datetime
from datetime import timedelta

In [190]:
file_name = choose_file()
start_date = end_date = get_date()
table, symbl = create_table(file_name)  # create pandas df
print '\nwork tables created...'
shares = read_shares(symbl, start_date, end_date)  # deliver shares from yahoo
print 'shares delivered...'
create_output(table, shares)  # create, reformat and write output xls
print 'output file created'



work tables created...
shares delivered...
output file created


In [187]:
shares

,symbl,Adj Close,Close,High,Low,Open,Current Date
0,ABT,61.720001,61.720001,64.160004,60.740002,61.000000,2018-01-24
1,BX,36.080002,36.080002,36.330002,35.680000,36.250000,2018-01-24
2,CMCSA,42.990002,42.990002,44.000000,41.570000,42.700001,2018-01-24
3,CVX,131.389999,131.389999,132.669998,131.070007,131.380005,2018-01-24
4,MDT,85.760002,85.760002,86.129997,85.160004,85.830002,2018-01-24
5,MS,57.240002,57.240002,57.730000,56.900002,57.209999,2018-01-24
6,PM,110.599998,110.599998,110.889999,109.559998,109.559998,2018-01-24
7,SUN,32.540001,32.540001,33.040001,32.419998,32.529999,2018-01-24
8,UN,58.240002,58.240002,58.320000,57.990002,58.279999,2018-01-24


In [162]:
symbl

array([u'CMCSA', u'SUN', u'CVX', u'MDT', u'ABT', u'BX', u'PM', u'UN', u'MS'], dtype=object)

In [163]:
panel = web.DataReader(symbl, 'yahoo', start_date, end_date)

In [164]:
aaa = panel.to_frame()

In [165]:
print start_date, end_date

2018-01-24 2018-01-24


In [166]:
aaa.columns

Index([u'Adj Close', u'Close', u'High', u'Low', u'Open', u'Volume'], dtype='object')

In [167]:
aaa

Adj Close       Close        High         Low        Open  \
Date       minor                                                               
2018-01-24 ABT     61.720001   61.720001   64.160004   60.740002   61.000000   
           BX      36.080002   36.080002   36.330002   35.680000   36.250000   
           CMCSA   42.990002   42.990002   44.000000   41.570000   42.700001   
           CVX    131.389999  131.389999  132.669998  131.070007  131.380005   
           MDT     85.760002   85.760002   86.129997   85.160004   85.830002   
           MS      57.240002   57.240002   57.730000   56.900002   57.209999   
           PM     110.599998  110.599998  110.889999  109.559998  109.559998   
           SUN     32.540001   32.540001   33.040001   32.419998   32.529999   
           UN      58.240002   58.240002   58.320000   57.990002   58.279999   
2018-01-23 ABT     59.230000   59.230000   59.570000   58.980000   59.520000   
           BX      35.830002   35.830002   36.020000   34.950001   35.000000   
           CMCSA   42.439999   42.439999   42.840000   41.959999   42.500000   
           CVX    131.020004  131.020004  132.559998  130.729996  132.380005   
           MDT     85.230003   85.230003   86.220001   84.980003   86.220001   
           MS      56.910000   56.910000   57.310001   56.650002   57.139999   
           PM     109.529999  109.529999  109.779999  108.400002  108.500000   
           SUN     32.410000   32.410000   32.880001   32.279999   32.599998   
           UN      57.910000   57.910000   58.169998   57.730000   57.910000   

                      Volume  
Date       minor              
2018-01-24 ABT    11704900.0  
           BX      5629000.0  
           CMCSA  48918800.0  
           CVX     6868200.0  
           MDT     5680200.0  
           MS      8403600.0  
           PM      4404100.0  
           SUN      490200.0  
           UN      1281200.0  
2018-01-23 ABT     8122100.0  
           BX      4987700.0  
           CMCSA  19155500.0  
           CVX     5747300.0  
           MDT     4962300.0  
           MS      9372200.0  
           PM      3778100.0  
           SUN      413700.0  
           UN      1075000.0

In [132]:
aaa.index

MultiIndex(levels=[[2018-01-24 00:00:00, 2018-01-23 00:00:00], [u'ABT', u'BX', u'CMCSA', u'CVX', u'MDT', u'MS', u'PM', u'SUN', u'UN']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8]],
           names=[u'Date', u'minor'])

In [114]:
aaa.index.levels[0][0]

Timestamp('2018-01-24 00:00:00')

In [106]:
aaa.columns

Index([u'Adj Close', u'Close', u'High', u'Low', u'Open', u'Volume'], dtype='object')

In [177]:
aaa.index.names

FrozenList([u'Date', u'minor'])

In [188]:
aaa.drop(start_date - timedelta(days=1) , level=0, axis=0)

Adj Close       Close        High         Low        Open  \
Date       minor                                                               
2018-01-24 ABT     61.720001   61.720001   64.160004   60.740002   61.000000   
           BX      36.080002   36.080002   36.330002   35.680000   36.250000   
           CMCSA   42.990002   42.990002   44.000000   41.570000   42.700001   
           CVX    131.389999  131.389999  132.669998  131.070007  131.380005   
           MDT     85.760002   85.760002   86.129997   85.160004   85.830002   
           MS      57.240002   57.240002   57.730000   56.900002   57.209999   
           PM     110.599998  110.599998  110.889999  109.559998  109.559998   
           SUN     32.540001   32.540001   33.040001   32.419998   32.529999   
           UN      58.240002   58.240002   58.320000   57.990002   58.279999   

                      Volume  
Date       minor              
2018-01-24 ABT    11704900.0  
           BX      5629000.0  
           CMCSA  48918800.0  
           CVX     6868200.0  
           MDT     5680200.0  
           MS      8403600.0  
           PM      4404100.0  
           SUN      490200.0  
           UN      1281200.0

In [149]:
from datetime import timedelta

In [151]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)

In [173]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)